In [ ]:
#@title Copyright 2024 Google LLC. { display-mode: "form" }
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Summary

This notebook demonstrates how to define, test, and deploy a Cloud Function.

# Setup

## Install Python packages

Uses the IPython magic [%%writefile](https://ipython.org/ipython-doc/3/interactive/magics.html#cellmagic-writefile) to create a file with names of the Python packages that will be installed.

In [ ]:
%%writefile requirements.txt
google-auth

Use the [pip](https://packaging.python.org/en/latest/key_projects/#pip) package installer to install Python packages in the Colab kernel.

In [ ]:
%%capture
!pip install -r requirements.txt

## Read Secrets

In order to avoid sharing user-specific information in the notebook, save the GCP project ID as a **Colab secret** named `EE_PROJECT_ID` before running the following cell. See [this guide](https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75) for information on how to use Colab secrets.

Note that you will need to grant access to Colab Secrets when this notebook is run.

In [ ]:
from google.colab import userdata

EE_PROJECT_ID = userdata.get('EE_PROJECT_ID')

# Define

Create a [Cloud Function definition file](https://cloud.google.com/functions/docs/writing#directory-structure-python) (`main.py`) with a function [entry point](https://cloud.google.com/functions/docs/writing#entry-point) (`hello_http()`):

In [ ]:
%%writefile main.py
from markupsafe import escape
import google.auth

credentials, project_id = google.auth.default();

def hello_http(request):
    """HTTP Cloud Function.
    Args:
        request (flask.Request): The request object.
        <https://flask.palletsprojects.com/en/1.1.x/api/#incoming-request-data>
    Returns:
        The response text, or any set of values that can be turned into a
        Response object using `make_response`
        <https://flask.palletsprojects.com/en/1.1.x/api/#flask.make_response>.
    """
    request_json = request.get_json(silent=True)
    request_args = request.args

    if request_json and "name" in request_json:
        name = request_json["name"]
    elif request_args and "name" in request_args:
        name = request_args["name"]
    else:
        name = "World"
    return f"Heya {escape(name)}, I'm a function running on projects/{project_id or 'unknown'}!"

# Test

Test out the cloud function locally.

In [ ]:
import json
import importlib
import main
importlib.reload(main) # Force reload of the main file.
from flask import Request

from google.colab import auth
auth.authenticate_user(project_id=EE_PROJECT_ID)

data = {
    "name": "Michael",
}

request = Request.from_values(
    headers={'Content-Type': 'application/json'},
    data=json.dumps(data),
    method="POST")

print(main.hello_http(request))

# Deploy

Once the function is working locally, deploy the function to Google Cloud.

In [ ]:
!gcloud functions deploy hello_http \
  --runtime python312 \
  --trigger-http \
  --allow-unauthenticated \
  --project $EE_PROJECT_ID

In [ ]:
print(f'https://console.cloud.google.com/run?project=vorgeo-training')

https://console.cloud.google.com/run?project=vorgeo-training